# 线性代数
## 依赖包

```shell
# https://pypi.org/project/PuLP/#files
python -m pip install -U git+https://github.com/coin-or/pulp
conda install -y scipy sympy
```

##  矩阵运算

$A=\begin{bmatrix} 3 & -1 & 2 \\ 1 & 5 & 7 \\ 5 & 4 & -3 \\ \end{bmatrix},$
$B=\begin{bmatrix} 7 & 5 & -4 \\ 1 & 1 & 9 \\ 3 & -2 & 1 \\ \end{bmatrix}$

### 加(减)法
A 与 B 均是同型矩阵，对应位置元素相加(减)

In [22]:
from sympy import Matrix

A = Matrix([[3, -1, 2], [1, 5, 7], [5, 4, -3]])
B = Matrix([[7, 5, -4], [1, 1, 9], [3, -2, 1]])
A + B

⎡10  4  -2⎤
⎢         ⎥
⎢2   6  16⎥
⎢         ⎥
⎣8   2  -2⎦

### 数乘
矩阵每个元素与该常数相乘

In [10]:
2 * A

⎡6   -2  4 ⎤
⎢          ⎥
⎢2   10  14⎥
⎢          ⎥
⎣10  8   -6⎦

### 乘法
C=AB
左边 A 的列数 与右边 B 的行数相等
A 第 i 行与 B 第 j 列对应元素相乘的和作为 C 第 i 行 j 列的元素

In [12]:
A * B

⎡26  10  -19⎤
⎢           ⎥
⎢33  -4  48 ⎥
⎢           ⎥
⎣30  35  13 ⎦

### 转置
行与列依次对换

In [14]:
A.T

⎡3   1  5 ⎤
⎢         ⎥
⎢-1  5  4 ⎥
⎢         ⎥
⎣2   7  -3⎦

### 逆矩阵
n 阶方阵可逆的充要条件是 r(A)=n，即 A 满秩

In [37]:
from sympy import eye

A_inv = eye(3)
if A.shape[0] == A.rank():
    A_inv = A.inv()
A_inv.evalf(4)

⎡0.2057   -0.02392  0.08134 ⎤
⎢                           ⎥
⎢-0.1818  0.09091   0.09091 ⎥
⎢                           ⎥
⎣0.1005   0.08134   -0.07656⎦

## 线性规划

$max \quad f=72x_1+64x_2 \\
s.t.
\begin{cases}
x_1 + x_2 \leq 50\\
12x_1 + 8x_2 \leq 480\\
3x_1 \leq 100\\
x_1 \geq 0, \quad x_2 \geq 0
\end{cases}
$

In [118]:
import pulp

model = pulp.LpProblem("demo", pulp.LpMaximize)
x1 = pulp.LpVariable('x1', lowBound=0, cat=pulp.const.LpContinuous)
x2 = pulp.LpVariable('x2', lowBound=0, cat=pulp.const.LpContinuous)
model += 72 * x1 + 64 * x2
model += x1 + x2 <= 50
model += 12 * x1 + 8 * x2 <= 480
model += 3 * x1 <= 100
model.solve()
print("x1={}".format(x1.varValue))
print('x2={}'.format(x2.varValue))
print('y={}'.format(pulp.value(model.objective)))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/tanghao/miniconda3/envs/learning/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/f9/y8qg28hs66g9nmh5pct38vfm0000gn/T/d735441b3fa44b529b29c0ab7f70b1ce-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/f9/y8qg28hs66g9nmh5pct38vfm0000gn/T/d735441b3fa44b529b29c0ab7f70b1ce-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 16 RHS
At line 20 BOUNDS
At line 21 ENDATA
Problem MODEL has 3 rows, 2 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-1) rows, 2 (0) columns and 4 (-1) elements
0  Obj -0 Dual inf 136 (2)
2  Obj 3360
Optimal - objective value 3360
After Postsolve, objective 3360, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 3360 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed fr

In [111]:
from sympy import Matrix, ones, diag

# material = Matrix([
#     [67.1, 29.2, 0.97, 2.01, 0.94, 0.66, 9.58, 3.5, 800],
#     [65.3, 27.4, 0.69, 6.64, 0.76, 0.23, 7.06, 2.1, 780],
#     [64.9, 28.05, 1.94, 2.49, 0.83, 0.61, 7.22, 2.9, 765],
#     [0, 0, 3.5, 43.8, 2.9, 0.02, 11.2, 11, 565],
#     [62.6, 5.6, 0.16, 3.7, 0.5, 0.06, 0, 0, 16]
# ])

# pct = Matrix([
#     [24, 28, 48, 1.5, 0.6]
# ])

material = Matrix([
    [.671, .292, .0097, .0201, .0094, .0066, .0958, .035, 800],
    [.653, .274, .0069, .0664, .0076, .0023, .0706, .021, 780],
    [.649, .2805, .0194, .0249, .0083, .0061, .0722, .029, 765],
])

pct = Matrix([
    [0.24, 0.28, 0.48]
])
material_w = material[0:3, 6:7]
r, c = material_w.shape
diag(pct.values(), unpack=True) * (ones(r, c) - material_w) * (1 / pct.dot((ones(r, c) - material_w)))

Matrix([
[0.235217606201712],
[0.282068624645561],
[0.482713769152728]])